# Prediction Mechanism 

## Import all relevant libraries and data

In [16]:
!pip install pandas
!pip install seaborn
!pip install keras
!pip install sklearn

In [17]:
import sys
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
#import tensorflow_docs as tfdocs
#import tensorflow_docs.modeling
#import tensorflow_docs.plots
#import tensorflow_docs as tfdocs
#import --upgrade tensorflow 
from keras.optimizers import Adam
from keras.utils import plot_model
from tensorflow import keras
from keras.models import Sequential, save_model, load_model
from keras.layers import LSTM, Dense, Dropout, GRU
from sklearn.preprocessing import MinMaxScaler


## Data Visualization

#### A file is chosen as a sample. The code is generalized for all such files.

In [ ]:
abl = pd.read_csv('BATA PA Equity.csv') 
abl.tail()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(abl.shape[0]), abl['Open'])
plt.xticks(range(0, abl.shape[0], 500), rotation=45)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Open', fontsize=18)
plt.show()

# Test-Train-Validation Split

#### The data is split into 3 parts. 

#### The first is the train part where we provide the model real prices as input and labels. It analyses the prices and attempts to predict the next day price. When predicted, the actual price for that day is given so it can learn accordingly.

#### The second is the test part, where we provide the model data it has not seen before and ask it to predict using this new data entirely.

#### The third part is the validation part. We keep some data never shown to the model in train and test parts and then ask it to predict this without ever seeing it.

In [108]:
def test_train_split(df, column):

    prices = df[column] #We choose to predict the price of the given column

    # 80-20 split into Train-Test

    train_data = prices[:int(len(prices)*0.7)]
    test_data = prices[int(len(prices)*0.7):int(len(prices)*0.8)]
    val_data = prices[int(len(prices)*0.8):]
    
    #Convert to ndarray

    train_data = np.asarray(train_data)
    test_data = np.asarray(test_data)
    val_data = np.asarray(val_data)
    
    #Transform data
    
    train_data = train_data.reshape(-1, 1)
    test_data = test_data.reshape(-1, 1)
    val_data = val_data.reshape(-1, 1)
    

    print(train_data.shape[0])
    print(test_data.shape[0])
    print(val_data.shape[0])

    
    return train_data, test_data, val_data
    
#train_data, test_data, val_data = test_train_split(abl, 'Open')

## Check distribution of data with scaling

#### A recurrent neural network works better when data is squished between a limit. We want to see if scaling our data this way has any change on its distribution. 

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Data Distribution')
ax1.boxplot(train_data, patch_artist=True)

tempScaler = MinMaxScaler()
scaled_data = tempScaler.fit_transform(train_data)

fig1, ax1 = plt.subplots()
ax1.set_title('Scaled Data Distribution')
ax1.boxplot(scaled_data, patch_artist=True)

#### We can see scaling our data does not change the distribution

## Prepare Train set

#### The data is prepared with the following methodology:

#### We take a window (set of days) to analyze the recent stock movemement. Based on that analysis, and the weight of historical learning (previous analyses), a prediction for  the window+1 day is made. The model compares the predicted price with the original price and adjusts its parameters. This is done iteratively, with a step of window+1 in each iteration. 

#### For example, for a window of 5, the model would look at day 1 to day 5 and predict day 6. Once day 6 is predicted, it would adjust parameters and then move onto the next window of day 7 to day 11 and predict day 12 and so on.

#### The days in the window are stored in x_train (the features). The days to be predicted are stored in y_train (labels). This results in two arrays, x_train and y_train which we would feed into the model

In [109]:
def train_prep(train_data, window, itr):
    scaler = MinMaxScaler((0, 1))
    scaled_train_data = scaler.fit_transform(train_data)

    x_train = [] #Features
    y_train = [] #Labels

    for i in range(window+itr, scaled_train_data.shape[0], window+1): #Make windows of 60 days for training until last day

        x_train.append(scaled_train_data[i-window:i]) #n day window to look back
        y_train.append(scaled_train_data[i:i+1, 0]) #predict for specified days

    x_train = np.array(x_train) 
    y_train = np.array(y_train)
    
        #print(x_train.shape, y_train.shape, scaler.scale_)
    
    return x_train, y_train, scaler.scale_
    

## Prepare test and validation set

#### Exactly the same way as train set, we will make the test set. The only difference is that the test set takes the last window from the train set so it can predict in continuation. 

In [110]:
def test_prep(test_data, train_data, window, itr):
    scaler = MinMaxScaler((0, 1))
    test_data = np.concatenate((train_data[:window], test_data), axis = 0) #Add last previous observations of train data to test
    scaled_test_data = scaler.fit_transform(test_data)

    x_test = []
    y_test = []

    for i in range(window+itr, test_data.shape[0], window+1):
        x_test.append(scaled_test_data[i-window:i])
        y_test.append(scaled_test_data[i:i+1, 0])

    x_test = np.array(x_test)
    y_test = np.array(y_test)
        
        #print(x_test.shape, y_test.shape, scaler.scale_)
    return x_test, y_test, scaler.scale_
    
#x_test, y_test, test_scale = test_prep(test_data, train_data, 5)

In [111]:
def val_prep(val_data, test_data, window, itr):
    scaler = MinMaxScaler((0, 1))
    val_data = np.concatenate((test_data[:window], val_data), axis = 0) #Add last previous observations of train data to test
    scaled_val_data = scaler.fit_transform(val_data)

    x_val = []
    y_val = []

    for i in range(window+itr, val_data.shape[0], window+1):
        x_val.append(scaled_val_data[i-window:i])
        y_val.append(scaled_val_data[i:i+1, 0])

    x_val = np.array(x_val)
    y_val = np.array(y_val)
        
    return x_val, y_val, scaler.scale_
    
#x_val, y_val, val_scale = val_prep(val_data, test_data, 5)

## Building the GRU

#### We chose the GRU model as it performed the best on our problem. The architecture is such that there are two layers of GRU. The first layer takes the input as we defined in train and test sets. It then forwards its results to the second layer which works similarly. Both layers are activated by ReLU on the output to ensure non-linearity of our problem. 

In [112]:
def create_model(x_train, units_1, units_2):

    model = Sequential()

    model.add(GRU(units = units_1, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))

    model.add(GRU(units = units_2, activation = 'relu'))

    model.add(Dense(units = 1))
    
    return model
    
#model = create_model(x_train, 16, 32)

#model.summary()

#plot_model(model, to_file='model.png')

#### ADAM is an optimizer used to optimize the Stochastic Gradient Descent used to minimize the error the model makes. Mean Squared Error is the error or loss function used to tell the model how to minimize the error in each step. The number of epochs is defined as how many times the model would need to go through the whole data in order to reach the minimum point of the loss. We have chosen the number at which we are sure the loss becomes constant and does not drop significiantly anymore. The batch size is defined as how many training examples should it put in one batch since the parameters are adjusted after each batch.

In [113]:
def compile_model(model, x_train, y_train, x_test, y_test):

    model.compile(optimizer='adam', loss= 'mean_squared_error')
    history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data = (x_test, y_test), verbose = 1)
    return history


## Testing

#### This is the part where we test our results. y_hat is an array which holds the predictions done on the test dataset we created earlier. After we have y_hat, we revert the scale of data to get original prices once again. These prices are then plotted to closely see how our model is behaving.

In [114]:
def plot_comparison(y_test, y_hat):

    plt.figure(figsize=(14,5))
    plt.plot(y_test[:], color = 'red', label = 'Real Stock Price')
    plt.plot(y_hat[:], color = 'blue', label = 'Predicted Stock Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()
    
#plot_comparison(y_test_unscaled, y_hat_unscaled)

In [115]:
KSE100 = pd.read_csv('KSE100.csv')
filtered = KSE100['Ticker'].tolist()
name = []
for i in range(len(filtered)):
    name.append(str(filtered[i]) + ".csv")

In [116]:
a=name[1]
name[1]=name[0]
name[0]=a
print(name)

['UBL PA Equity.csv', 'HBL PA Equity.csv', 'HUBC PA Equity.csv', 'MCB PA Equity.csv', 'OGDC PA Equity.csv', 'LUCK PA Equity.csv', 'ENGRO PA Equity.csv', 'PPL PA Equity.csv', 'FFC PA Equity.csv', 'PSO PA Equity.csv', 'DAWH PA Equity.csv', 'DGKC PA Equity.csv', 'POL PA Equity.csv', 'EFERT PA Equity.csv', 'KAPCO PA Equity.csv', 'NBP PA Equity.csv', 'NESTLE PA Equity.csv', 'SEARL PA Equity.csv', 'BAHL PA Equity.csv', 'FCCL PA Equity.csv', 'MLCF PA Equity.csv', 'KEL PA Equity.csv', 'PSEL PA Equity.csv', 'PIBTL PA Equity.csv', 'PKGS PA Equity.csv', 'NML PA Equity.csv', 'BAFL PA Equity.csv', 'MARI PA Equity.csv', 'PAEL PA Equity.csv', 'FFBL PA Equity.csv', 'KTML PA Equity.csv', 'ABOT PA Equity.csv', 'ABL PA Equity.csv', 'TRG PA Equity.csv', 'EFUG PA Equity.csv', 'FCEPL PA Equity.csv', 'INDU PA Equity.csv', 'NRL PA Equity.csv', 'MUREB PA Equity.csv', 'FATIMA PA Equity.csv', 'HCAR PA Equity.csv', 'NATF PA Equity.csv', 'HASCOL PA Equity.csv', 'PIOC PA Equity.csv', 'HMB PA Equity.csv', 'PAKT PA E

In [117]:
global mean_acc
mean_acc = []
def main(df, column, itre, dates):  
    dfs = pd.DataFrame(columns = ['Date', 'Predicted Price'])
    dates1 = dates[int(len(dates)*0.7):]
    dfs['Date'] = dates1
    for itr in range (0,itre):
        train_data, test_data, val_data = test_train_split(df, column)
        tempScaler = MinMaxScaler()
        scaled_data = tempScaler.fit_transform(train_data)
        x_train, y_train, scale_train = train_prep(train_data, 5, itr)
        x_test, y_test, test_scale = test_prep(test_data, train_data, 5, itr)
        x_val, y_val, val_scale = val_prep(val_data, test_data, 5, itr)
        model = create_model(x_train, 16, 32)
        history = compile_model(model, x_train, y_train, x_test, y_test)
        y_hat = model.predict(x_test)
        y_hat_unscaled = y_hat/test_scale[0]
        y_test_unscaled = y_test/test_scale[0]
        y_hat_val = model.predict(x_val)
        y_hat_val_unscaled = y_hat_val/val_scale[0]
        y_val_unscaled = y_val/val_scale[0] 
        
        j = int(len(dates)*0.7)+itr
        for i in range(len(y_hat)):
            dfs['Predicted Price'][j] = y_hat[i][0]
            j+=6
        j = int(len(dates)*0.8)+itr
        for i in range(len(y_hat_val)):
            dfs['Predicted Price'][j] = y_hat_val[i][0]
            j+=6
    dfs = dfs.iloc[40:]
    #print('complete')
    return dfs

def companies(names):
    for i in names:
        df = pd.read_csv('C:/Users/Legion/Desktop/Deep-PSX-Kaavish-main/Deep-PSX-Kaavish-main Updated/Deep-PSX-Kaavish-main/Companies/'+i) 
        dates = df['Date']
        dfs = main(df, 'Close', 6, dates)
        dfs.to_csv(path_or_buf = 'C:/Users/Legion/Desktop/Deep-PSX-Kaavish-main/Deep-PSX-Kaavish-main Updated/Deep-PSX-Kaavish-main/Companies predictions/predicted '+i, encoding='utf-8',  sep = ',', index = False) #Save dataframe as CSV
companies(name)

2778
397
794
Epoch 1/10
15/15 [==============================] - 0s 21ms/step - loss: 0.1391 - val_loss: 0.2758
Epoch 2/10
15/15 [==============================] - 0s 4ms/step - loss: 0.0676 - val_loss: 0.0974
Epoch 3/10
15/15 [==============================] - 0s 3ms/step - loss: 0.0391 - val_loss: 0.0378
Epoch 4/10
15/15 [==============================] - 0s 3ms/step - loss: 0.0234 - val_loss: 0.0291
Epoch 5/10
15/15 [==============================] - 0s 3ms/step - loss: 0.0086 - val_loss: 0.0056
Epoch 6/10
15/15 [==============================] - 0s 3ms/step - loss: 7.9608e-04 - val_loss: 0.0044
Epoch 7/10
15/15 [==============================] - 0s 3ms/step - loss: 4.0592e-04 - val_loss: 0.0041
Epoch 8/10
15/15 [==============================] - 0s 3ms/step - loss: 2.2524e-04 - val_loss: 0.0038
Epoch 9/10
15/15 [==============================] - 0s 3ms/step - loss: 1.9726e-04 - val_loss: 0.0038
Epoch 10/10
15/15 [==============================] - 0s 3ms/step - loss: 1.8742e-04 - va

13/13 [==============================] - 0s 4ms/step - loss: 3.9811e-04 - val_loss: 0.0048
Epoch 9/10
13/13 [==============================] - 0s 3ms/step - loss: 3.2686e-04 - val_loss: 0.0041
Epoch 10/10
13/13 [==============================] - 0s 3ms/step - loss: 2.4249e-04 - val_loss: 0.0041
suffwan randi
1
2417
346
691
Epoch 1/10
13/13 [==============================] - 0s 23ms/step - loss: 0.1949 - val_loss: 0.3037
Epoch 2/10
13/13 [==============================] - 0s 4ms/step - loss: 0.1093 - val_loss: 0.1542
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 0.0479 - val_loss: 0.0479
Epoch 4/10
13/13 [==============================] - 0s 3ms/step - loss: 0.0215 - val_loss: 0.0096
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 0.0127 - val_loss: 0.0117
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 0.0045 - val_loss: 0.0031
Epoch 7/10
13/13 [==============================] - 0s 3ms/step - loss: 4.9264e-04 - v

Epoch 7/10
19/19 [==============================] - 0s 3ms/step - loss: 1.3750e-04 - val_loss: 5.5975e-04
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 9.6339e-05 - val_loss: 4.9494e-04
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 7.7112e-05 - val_loss: 4.6968e-04
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - loss: 6.6450e-05 - val_loss: 4.4525e-04
suffwan randi
3
3610
516
1032
Epoch 1/10
19/19 [==============================] - 0s 17ms/step - loss: 0.1170 - val_loss: 0.4061
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0662 - val_loss: 0.2120
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0363 - val_loss: 0.0789
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0149 - val_loss: 0.0198
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0012 - val_loss: 0.0032
Epoch 6/10
19/19 [==============================] - 0s

19/19 [==============================] - 0s 3ms/step - loss: 0.0276 - val_loss: 0.1417
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0162 - val_loss: 0.0554
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0030 - val_loss: 0.0048
Epoch 6/10
19/19 [==============================] - 0s 3ms/step - loss: 4.1609e-04 - val_loss: 0.0037
Epoch 7/10
19/19 [==============================] - 0s 3ms/step - loss: 1.7877e-04 - val_loss: 0.0038
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 1.2620e-04 - val_loss: 0.0038
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 9.9390e-05 - val_loss: 0.0036
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - loss: 7.7945e-05 - val_loss: 0.0035
suffwan randi
1
3610
516
1032
Epoch 1/10
19/19 [==============================] - 0s 17ms/step - loss: 0.0397 - val_loss: 0.1828
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 0.019

16/16 [==============================] - 0s 3ms/step - loss: 1.9745e-04 - val_loss: 6.9543e-04
Epoch 10/10
16/16 [==============================] - 0s 3ms/step - loss: 1.8385e-04 - val_loss: 6.8555e-04
suffwan randi
4
2970
425
849
Epoch 1/10
16/16 [==============================] - 0s 20ms/step - loss: 0.1555 - val_loss: 0.4181
Epoch 2/10
16/16 [==============================] - 0s 4ms/step - loss: 0.0985 - val_loss: 0.2651
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0516 - val_loss: 0.1182
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0204 - val_loss: 0.0258
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 0.0038 - val_loss: 9.1732e-04
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 4.9869e-04 - val_loss: 7.3123e-04
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 1.8378e-04 - val_loss: 6.1823e-04
Epoch 8/10
16/16 [==============================] - 0s 4ms/ste

...

Epoch 8/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0134 - val_loss: 0.0335
Epoch 9/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0075 - val_loss: 0.0248
Epoch 10/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0028 - val_loss: 0.0204
suffwan randi
0
1304
186
373
Epoch 1/10
7/7 [==============================] - 0s 42ms/step - loss: 0.1731 - val_loss: 0.2748
Epoch 2/10
7/7 [==============================] - 0s 5ms/step - loss: 0.1377 - val_loss: 0.2220
Epoch 3/10
7/7 [==============================] - 0s 5ms/step - loss: 0.1052 - val_loss: 0.1683
Epoch 4/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0721 - val_loss: 0.1147
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0447 - val_loss: 0.0664
Epoch 6/10
7/7 [==============================] - 0s 5ms/step - loss: 0.0255 - val_loss: 0.0352
Epoch 7/10
7/7 [==============================] - 0s 4ms/step - loss: 0.0179 - val_loss: 0.0241
Epoch 8/1

suffwan randi
1
3610
516
1032
Epoch 1/10
19/19 [==============================] - 0s 17ms/step - loss: 0.0481 - val_loss: 0.3332
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0212 - val_loss: 0.1605
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0085 - val_loss: 0.0537
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0013 - val_loss: 0.0048
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - loss: 4.4114e-04 - val_loss: 0.0036
Epoch 6/10
19/19 [==============================] - 0s 3ms/step - loss: 2.5831e-04 - val_loss: 0.0021
Epoch 7/10
19/19 [==============================] - 0s 4ms/step - loss: 2.1394e-04 - val_loss: 0.0017
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 1.7746e-04 - val_loss: 0.0016
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 1.8680e-04 - val_loss: 0.0016
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - 

19/19 [==============================] - 0s 3ms/step - loss: 1.0619e-04 - val_loss: 5.0276e-04
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 8.8430e-05 - val_loss: 3.9993e-04
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 8.3284e-05 - val_loss: 4.2242e-04
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - loss: 8.1691e-05 - val_loss: 4.3601e-04
suffwan randi
5
complete
2511
359
718
Epoch 1/10
14/14 [==============================] - 0s 23ms/step - loss: 0.1134 - val_loss: 0.3781
Epoch 2/10
14/14 [==============================] - 0s 4ms/step - loss: 0.0615 - val_loss: 0.1782
Epoch 3/10
14/14 [==============================] - 0s 3ms/step - loss: 0.0307 - val_loss: 0.0619
Epoch 4/10
14/14 [==============================] - 0s 3ms/step - loss: 0.0182 - val_loss: 0.0363
Epoch 5/10
14/14 [==============================] - 0s 3ms/step - loss: 0.0063 - val_loss: 0.0141
Epoch 6/10
14/14 [==============================] - 0s 3m

Epoch 6/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0037 - val_loss: 0.0084
Epoch 7/10
19/19 [==============================] - 0s 3ms/step - loss: 4.9043e-04 - val_loss: 0.0035
Epoch 8/10
19/19 [==============================] - 0s 3ms/step - loss: 2.5784e-04 - val_loss: 0.0033
Epoch 9/10
19/19 [==============================] - 0s 3ms/step - loss: 2.0676e-04 - val_loss: 0.0030
Epoch 10/10
19/19 [==============================] - 0s 3ms/step - loss: 2.0014e-04 - val_loss: 0.0030
suffwan randi
1
3610
516
1032
Epoch 1/10
19/19 [==============================] - 0s 17ms/step - loss: 0.0387 - val_loss: 0.0708
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0089 - val_loss: 0.0250
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: 0.0037 - val_loss: 0.0053
Epoch 4/10
19/19 [==============================] - 0s 3ms/step - loss: 5.2611e-04 - val_loss: 0.0022
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - 

10/10 [==============================] - 0s 4ms/step - loss: 0.0110 - val_loss: 0.0086
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0038
Epoch 7/10
10/10 [==============================] - 0s 4ms/step - loss: 5.5317e-04 - val_loss: 0.0035
Epoch 8/10
10/10 [==============================] - 0s 4ms/step - loss: 7.5622e-04 - val_loss: 0.0026
Epoch 9/10
10/10 [==============================] - ETA: 0s - loss: 5.3155e-0 - 0s 4ms/step - loss: 4.9805e-04 - val_loss: 0.0026
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 3.9638e-04 - val_loss: 0.0025
suffwan randi
3
1789
256
512
Epoch 1/10
10/10 [==============================] - 0s 29ms/step - loss: 0.1628 - val_loss: 0.5134
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 0.1254 - val_loss: 0.3983
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0952 - val_loss: 0.2956
Epoch 4/10
10/10 [==============================] - 0s